## Table of Contents

<div class="Segmenting and Clustering Neighborhoods in Toronto" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Start by creating a new Notebook for this assignment.</a>

2. <a href="#item2">obtain the data</a>

3. <a href="#item3">To create the above dataframe:</a>

4. <a href="#item4">Submit a link to your Notebook on your Github repository</a>

</font>
</div>

### step 1: obtain the data

In [3]:
pip install bs4

     |████████████████████████████████| 102kB 11.9MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 24.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [55]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests


#!wget -q -O 'customer_segmentation1.csv'  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
#print('Data downloaded!')
#customers_df = pd.read_csv('customer_segmentation1.csv')
#customers_df.head()

In [20]:
from bs4 import BeautifulSoup
import requests
import csv
import bs4 #用来解析网页，提取指定数据的 bs4里面有一个类，BeautifulSoup，然后通过这个类将网页html格式字符串生成一个对象，然后通过对象的方法来进行查找指定元素
import lxml
from lxml import etree
 
#检查url地址
def check_link(url):
    try:   
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print('无法链接服务器！！！')
 
 
#爬取资源
def get_contents(ulist,rurl):
    soup = BeautifulSoup(rurl,'lxml')
    trs = soup.find_all('tr')
    for tr in trs:
        ui = []
        for td in tr:
            ui.append(td.string)
        ulist.append(ui)

    
#保存资源
def save_contents(urlist):
    with open("Toronto-code.csv",'w') as f:
        writer = csv.writer(f)
        writer.writerow([['1'],['2'],['3']])
        for i in range(len(urlist)):
            if i>288:   #该表格暂时只有288行
                break;
            #print(i,urlist[i][1],urlist[i][3],urlist[i][5])
            writer.writerow([urlist[i][1],urlist[i][3],urlist[i][5]])
            
 
def main():
    urli = []
    url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    rs = check_link(url)
    get_contents(urli,rs)
    #print(len(urli))
    #print(urli)
    save_contents(urli)
main()

In [56]:
df_code= pd.read_csv('Toronto-code.csv')
df_code=df_code[1:288]
df_code.columns=list(['Postcode','Borough','Neighbourhood'])
df_code.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,NaN
4,M4A,North York,NaN
5,M5A,Downtown Toronto,NaN
6,M5A,Downtown Toronto,NaN
7,M6A,North York,NaN
8,M6A,North York,NaN
9,M7A,Queen's Park,Not assigned\n
10,M8A,Not assigned,Not assigned\n


### step 2 wragging the data

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [65]:
df = df_code[df_code['Borough']!='Not assigned']
df=df.reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')
df['Borough']=df['Borough'].str.replace('\n','')
df=df.sort_values(['Postcode','Borough'])

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [67]:
df.loc[df['Neighbourhood'].isnull(),'Neighbourhood']=df[df['Neighbourhood'].isnull()]['Borough']
#grouped.head(20)
df.head()

,index,Postcode,Borough,Neighbourhood
8,12,M1B,Scarborough,Scarborough
9,13,M1B,Scarborough,Scarborough
21,28,M1C,Scarborough,Scarborough
22,29,M1C,Scarborough,Scarborough
23,30,M1C,Scarborough,Scarborough


### These two rows will be combined into one row with the neighborhoods separated with a comma

In [68]:
grouped = df.groupby(['Postcode','Borough'])
result = grouped['Neighbourhood'].unique()
result2 = result.reset_index()
result2['Neighbourhood'] = result2['Neighbourhood'].apply(lambda x: ';'.join(x))
result2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Scarborough
1,M1C,Scarborough,Scarborough
2,M1E,Scarborough,Guildwood;Scarborough
3,M1G,Scarborough,Scarborough
4,M1H,Scarborough,Cedarbrae


In [70]:
result2.shape

(103, 3)